# HDFS

HDFS is het distributed file system van Hadoop dat de basis vormt voor een breed gamma van applicaties, waaronder MapReduce.
Dit framework maakt het mogelijk om niet-gespecialiseerde hardware te gebruiken om eenvoudig datacenters op te zetten of rekenclusters te beheren.
HDFS bereikt dit doel op een aantal manieren:
* Ten eerste wordt een file opgedeeld in verschillende blokken. Deze worden verdeeld over verschillende computers zodat code meerdere delen van het bestand kan gebruiken in parallel om zo de benodigde rekenkracht te verdelen over meerdere toestellen.
* Daarnaast worden de blokken ook gedupliceerd om zo fout-toleranter te zijn in het geval van een crash (hardware of software), stroomstoring, netwerkonderbreking.

Om dit te bereiken gebruikt Hadoop een Master-Slave architectuur dat bestaat uit een enkele namenode (master) en meerdere datanodes (slaves).
De namenode houdt bij hoeveel datanodes er actief zijn, welke blokken ze hebben en welke blokken bij welke file horen.
Indien er een datanode crasht gaat deze server dit detecteren en dit oplossen door de nodige blokken te kopieren van een andere datanode zodat er steeds voldoende kopies in het systeem aanwezig zijn.
Bij elke actie die uitgevoerd moet worden in het HDFS moet er steeds gevraagd worden aan de namenode welke blokken op welke datanodes we nodig hebben voor de gewenste file uit te lezen of code voor uit te voeren.
Het is dus duidelijk dat deze namenode een single-point-of-failure is wat ideaal is voor de availability van de cluster.
Dit kan opgelost worden door HDFS te runnen in een high-availability mode wat ervoor zorgt dat er steeds een backup aanwezig is voor de namenode die zeer snel de werking kan overnemen.
Deze structuur maakt het eenvoudig om aan horizontal scaling te doen door extra servers toe te voegen zonder dat er downtime is voor de hele cluster.

Dit resulteer in de volgende kenmerken van HDFS:
* High Throughput
* Scalability
* High Availability
* Data Reliability
* Fault Tolerance

## Starten en stoppen van de cluster

De cluster kan gestart worden door de volgende commando's uit te voeren in de commandline:
* start-dfs.sh
* start-yarn.sh

Eenmaal dat de cluster gestart is kunnen de lopende java applicaties gecontroleerd worden door middel van het commando jps.
Dit kan ook uitgevoerd worden vanuit deze jupyter notebook door onderstaande code-cell uit te voeren.

In [ ]:
!start-dfs.sh
!start-yarn.sh
#!start-all.sh   # combinatie van de twee bovenstaande
!jps

Ook heeft de namenode een website waar naartoe kan gegaan worden om meer informatie over de cluster te bekijken. De standaardurl is [http://localhost:9870](http://localhost:9870).

Om de cluster terug te stoppen moeten in de terminal of de notebook de volgende commando's uitgevoerd worden.
* stop-dfs.sh
* stop-yarn.sh


In [ ]:
#!stop-yarn.sh
#!stop-dfs.sh
#!stop.all.sh  # combinatie van de twee bovenstaande

## Communiceren met het HDFS

Er zijn verschillende manieren om dit distributed bestandssysteem te gebruiken.
Ten eerste kan je gebruik maken van een command line interface (CLI) om bestanden uit te lezen, op te laden, ...
Daarnaast zijn er ook wrappers voor verschillende talen die toelaten om rechtstreeks vanuit code te interageren met het HDFS.
De voordelen van deze wrappers over een CLI zijn:
* Flexibiler om te interageren in applicaties.
* Gebruiksvriendelijker en gemakkelijker om te automatiseren.
* Eenvoudiger in onderhoud en te debuggen
* Volledige functionaliteit van een programmeertaal kan gebruikt worden zoals OO.

### Instantiering van een client

Veel verschillende talen beschikken over een wrapper om te communiceren met een HDFS.
In deze cursus gaan we gebruik maken van [de pydoop](https://crs4.github.io/pydoop/index.html) wrapper.
De eerste stap in het gebruik van de wrapper is te bepalen hoe de namenode van het hdfs gevonden kan worden.
Dit gebeurt als volgt:

In [1]:
import pydoop.hdfs as hdfs

# ref to local fs
localFs = hdfs.hdfs(host='')

# ref to hadoop cluster
client = hdfs.hdfs(host='localhost', port=9000)

client.capacity()


2023-02-09 16:43:46,893 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


30994796544

### Aanmaken van files en directories

Om nu bestanden en folders aan te maken op dit distributed file systeem kunnen onderstaande functies gebruikt worden.

In [8]:
#!hdfs dfs -test -d test
#!hdfs dfs -mkdir -p test3

In [5]:
client.set_working_directory('test')
if not client.exists('test2'):
    client.create_directory('test2')   # dit wordt geplaatst onder user/bigdata/

In [12]:
#!hadoop fs -put davinci_notebooks.txt testbook.txt

In [13]:
#!hadoop fs -get testbook.txt book2.txt

In [11]:
localFs.copy("davinci_notebooks.txt", client, "book 1.txt")   # from {fs} copy {file} to {fs} en noem het {file2}
        # 1 replica -> er is maar 1 node dus kan hij niet zijn standaardgedrag van 3 replica's uitvoeren
        # block size is 128 MB > file-size dus is er 1 blok
client.copy("book 1.txt", localFs, "book 1.txt") 

client.move("book 1.txt", localFs, "book 1.txt") # origineel is verwijderd

### Bekijken van het filesysteem

In [17]:
client.list_directory('..')
client.used()

171483136

In [19]:
# read file line by line from the distributed fily system
f = client.open_file('../testbook.txt')
for i in range(5):
    print(f.readline())

b'The Project Gutenberg EBook of The Notebooks of Leonardo Da Vinci, Complete\n'
b'by Leonardo Da Vinci\n'
b'(#3 in our series by Leonardo Da Vinci)\n'
b'\n'
b'Copyright laws are changing all over the world. Be sure to check the\n'


In [20]:
!hdfs dfs -cat testbook.txt | head -n 5

The Project Gutenberg EBook of The Notebooks of Leonardo Da Vinci, Complete
by Leonardo Da Vinci
(#3 in our series by Leonardo Da Vinci)

Copyright laws are changing all over the world. Be sure to check the
cat: Unable to write to output stream.


### Aanpassen van het filesysteem

In [21]:
client.rename("../testbook.txt", "../book.txt")

In [22]:
client.delete('../test3')   # remove directory

In [23]:
client.set_working_directory('..')

In [27]:
#!hdfs dfs -rm book.txt    #remove file
!hdfs dfs -rm -r test       # remove directory

rm: `test': No such file or directory


In [24]:
client.delete('book.txt')# remove file

### Oefening

Schrijf python code dat controleert of een directory bestaat in het hdfs.
Indien nee wordt de directory aangemaakt.
Indien ja, worden alle files in de directory verwijderd om van een lege directory te starten.
Upload daarna een tekst-bestand naar keuze.